In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(x_train,  y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.
x_test = x_test / 255.

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
#Add the channel dimension (Last dimension)
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [5]:
#Create a class called Mymod which will inherit from the Model class using super()
class Mymod(Model):
  #Define all the layers in the init function
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten  = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  #Define all the layer computations in call function
  #x is the arguments you pass to the model (The model inputs)
  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

In [6]:
model = Mymod()

In [8]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam()

In [9]:
#Calculates the mean of the loss (Which you will pass to train_loss)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

In [17]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    #Make a prediction
    preds = model(images)
    #Calculate the loss
    loss = loss_obj(labels, preds)
    #Calculate the gradient of loss with respect to the weights & biases (the trainable variables)
    grads = tape.gradient(loss, model.trainable_variables)
    #Update the model weights & biases using the computed gradients
    opt.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_acc(labels, preds)

In [18]:
@tf.function
def test_step(images, labels):
  preds = model(images)
  t_loss = loss_obj(labels, preds)

  test_loss(t_loss)
  test_acc(labels, preds)

In [19]:
epochs = 5
for epoch in range(epochs):
  #Reset the metrics at the start of every epoch
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  for imgs, labels in train_ds:
    train_step(imgs, labels)

  for test_imgs, test_labels in test_ds:
    test_step(test_imgs, test_labels)

  #Obtain the results of the metrics using .result()
  print(f'Epochs {epoch+1}, Loss: {train_loss.result() *100}, Accuracy: {train_acc.result() * 100}, Test Loss: {test_loss.result()}, Test Accuracy: {test_acc.result()*100}')

Epochs 1, Loss: 0.5540868639945984, Accuracy: 99.82499694824219, Test Loss: 0.059485260397195816, Test Accuracy: 98.5199966430664
Epochs 2, Loss: 0.46895095705986023, Accuracy: 99.86000061035156, Test Loss: 0.06603164970874786, Test Accuracy: 98.61000061035156
Epochs 3, Loss: 0.34416118264198303, Accuracy: 99.88500213623047, Test Loss: 0.06767413020133972, Test Accuracy: 98.5199966430664
Epochs 4, Loss: 0.229197159409523, Accuracy: 99.92500305175781, Test Loss: 0.08756589144468307, Test Accuracy: 98.31999969482422
Epochs 5, Loss: 0.25664085149765015, Accuracy: 99.91166687011719, Test Loss: 0.09407512098550797, Test Accuracy: 98.2699966430664
